In [3]:
import pandas as pd
from itertools import product
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser


In [4]:
train = pd.read_csv("../data/train.csv")

In [5]:
train_data = []
for q,a in list(product([f"질문_{x}" for x in range(1,3)],[f"답변_{x}" for x in range(1,6)])):
    for i in range(len(train)):
        train_data.append(
            "질문: "+ train.at[i,q] + " 답변 : " + train.at[i,a]
        )

In [6]:
train_data[:5]

['질문: 면진장치가 뭐야? 답변 : 면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 격리장치입니다.',
 '질문: 내진설계의 종류 좀 알려줘 답변 : 내진 설계의 종류로 내진구조, 제진구조, 면진구조가 있습니다.',
 '질문: 철골구조의 장점이 뭐야? 답변 : 철골구조는 건물의 외벽에는 그다지 하중이 걸리지 않기 때문에 고층 건물의 건축이 가능한 것이 장점입니다.',
 '질문: 철골철근 콘크리트 구조가 뭐야? 답변 : 철근철골콘크리트는 철골과 철근, 그리고 콘크리트를 함께 사용하는 건축 구조입니다. 철골은 강하고 가벼운 소재로, 높은 내구성과 강도를 가지고 있어 대규모 공간이나 강성이 요구되는 대형 고층건물에 적합합니다. 그러나 화재에 취약하고 녹이 슬면 강성이 떨어지는 단점이 있습니다. 이를 보완하기 위해 철근과 콘크리트를 섞어 사용하는 방식이 개발되었습니다. 철근콘크리트는 철근이 인장력을, 콘크리트가 압축력을 담당하여 강성을 가지며 콘크리트의 열전도가 낮은 재료상 특징을 이용해 서로의 단점을 보완하도록 결합한 구조입니다. 내화성, 내구성, 내진성능이 뛰어나지만 자중이 크고 시공과정이 복잡하며 공사기간이 길고 균일한 시공이 불가능하다는 단점이 있습니다. 철근철골콘크리트는 이 두 가지 구조의 장점을 살려 만든 것으로, 철골 뼈대로 구조를 세우고 그 주변을 철근과 콘크리트로 감싸거나, 철근과 콘크리트로 구조를 만들고 그 위에 철골을 올리는 형태로 만들어집니다. 이렇게 함으로써 철골의 내구성과 강도를 높이면서도 화재에 대한 안전성을 높일 수 있습니다.',
 '질문: 철골구조는 어떤 방식이 있어? 답변 : 철골구조는 일반철골구조와 경량철골구조가 있습니다.']

In [16]:
pd.DataFrame(train_data).to_csv("../data/train_data.csv",index=False,encoding='utf-8')

In [7]:
loader = CSVLoader(file_path='../data/train_data.csv',encoding='utf-8')
data = loader.load()

In [13]:
data[1]

Document(page_content='0: 질문: 내진설계의 종류 좀 알려줘 답변 : 내진 설계의 종류로 내진구조, 제진구조, 면진구조가 있습니다.', metadata={'source': '../data/train_data.csv', 'row': 1})

In [26]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import LlamaForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=False,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype="float16",
        )

model_id = "beomi/llama-2-ko-7b"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = LlamaForCausalLM.from_pretrained(model_id, quantization_config=bnb_config).to(0)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512,device=0)
hf = HuggingFacePipeline(pipeline=pipe)

tokenizer_config.json:   0%|          | 0.00/842 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.55M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/606 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

...

model-00014-of-00015.safetensors:   0%|          | 0.00/742M [00:00<?, ?B/s]

model-00015-of-00015.safetensors:   0%|          | 0.00/380M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 172.00 MiB. GPU 0 has a total capacity of 23.69 GiB of which 30.06 MiB is free. Process 9426 has 7.93 GiB memory in use. Process 36053 has 15.73 GiB memory in use. Of the allocated memory 15.41 GiB is allocated by PyTorch, and 21.06 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [14]:
modelPath = "distiluse-base-multilingual-cased-v1"
model_kwargs = {'device':'cuda'}
encode_kwargs = {'normalize_embeddings': False}

embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [15]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: DistilBertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Dense({'in_features': 768, 'out_features': 512, 'bias': True, 'activation_function': 'torch.nn.modules.activation.Tanh'})
), model_name='distiluse-base-multilingual-cased-v1', cache_folder=None, model_kwargs={'device': 'cuda'}, encode_kwargs={'normalize_embeddings': False}, multi_process=False, show_progress=False)

In [17]:
db = FAISS.from_documents(data, embedding=embeddings)

In [20]:
db.save_local("faiss_index")

In [21]:
retriever = db.as_retriever(search_kwargs={"k": 4})

In [22]:
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7fc13ea91d20>, search_kwargs={'k': 4})

In [23]:
template = """마지막에 질문에 답하려면 다음과 같은 맥락을 사용합니다.

{context}

질문: {question}

유용한 답변:"""

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [24]:
custom_rag_prompt = PromptTemplate.from_template(template)

In [25]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | hf
    | StrOutputParser()
)


NameError: name 'hf' is not defined